In [29]:
#데이터 세트 준비 
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data') # 판다스로 csv읽어오고
data = wine[['alcohol', 'sugar', 'pH']].to_numpy() # 넘파이 데이터 프레임으로 data에 저장
target = wine['class'].to_numpy()  #  class는 타겟 값으로 저장

In [30]:
# 훈련세트 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2 , random_state =42
)

In [31]:
# 훈련세트를 -> sub훈련세트 , 검증세트로 나누기
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state = 42
)

In [32]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [33]:
# sub, 과 검증 세트로 모델을 만들고 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 훈련세트에 과대적합 되어있음
# 매개 변수 바꿔서 더 좋은 모델 찾아야함

0.9971133028626413
0.864423076923077


---
### 교차 검증 = cross_validate(
##### : 검증세트를 떼어 평가하는 과정을 여러번 반복 
-> 이 점수를 평균하여 최종 검증 점수 얻음
##### : 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있음
* 많은 데이터를 훈련에 사용하면 좋은 모델이 됨
* 그렇다고 검증 세트가 너무 조금이면 검증점수가 들쭉날쭉함
* 이럴 때 교차 검증을 이용하면
#### 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음
---
### 3-폴드 교차 검증(k-폴드 교차검증)
훈련 세트를 세 부분으로 나눠서 검증을 수행
* 주로 5폴드 10폴드 교차검증 많이 사용
* 이렇게 하면 데이터의 80~90퍼를 훈련에 사용할 수 있음

In [34]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
# fit time, score time  시간
# test score : 검증폴드 최종점수 

{'fit_time': array([0.01094127, 0.0070405 , 0.00735402, 0.00729752, 0.00789309]), 'score_time': array([0.00078106, 0.00069857, 0.00072217, 0.00071692, 0.00073862]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [35]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [36]:
# 교차 검증은 훈련세트를 섞지않음 
# StratifiedKFold = 교차검증에서 폴드를 어떻게 나눌지 결정
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [37]:
# 훈련세트를 섞은 후 10-폴드 교차검증 수행하기
splitter = StratifiedKFold(n_splits =10, shuffle = True, random_state = 42) # n_split =  몇 폴드 교차 검증인지
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


---
### 하이퍼 파라미터 튜닝
 
#### 그리드 서치
* GridSearchCV 클래스는 하이퍼 파라미터 탐색과 교차검증을 한 번에 수행
* 즉, 매개변수를 일일이 바꿔가며 교차 검증을 수행할 필요없음. 
* 원하는 매개변수 값을 나열하면 자동으로 교차검증을 수행해서 최상의 매개변수를 찾을 수 있음


In [38]:
from sklearn.model_selection import GridSearchCV  # 그리드 서치 임포트
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # min_impurity_decrease의 최적 값을 찾아보고자 함
                                                                             # 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듬

In [39]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1) # 그리드 서치 객체 만들기
                  # 탐색 대상 모델                          # params변수 전달 # 병렬 실행에 사용할 CPU 코어 수 지정, : -1은 모든 코어 사용

In [40]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

* 그리드 서치 cv 기본값은 5이므로 min_impurity_decrease rkqtakek 5폴드 교차검증을 수행
: 따라서 총 25개의 모델을 훈련함

* 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련 시킴
* 이 최적의 모델은 gs객체의 best_estimator_ 속성에 저장되어 있음 
* 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있음
* 교차 검증의 평균 점수는 cv_results_의 mean_test_score 에 저장

In [41]:
dt = gs.best_estimator_ # 이 최적의 모델은 gs객체의 best_estimator_ 속성에 저장되어 있음 
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있음


In [42]:
print(gs.best_params_) # 가장 좋은 값으로 0.0001이 선택됨

{'min_impurity_decrease': 0.0001}


In [43]:
print(gs.cv_results_['mean_test_score']) # 교차 검증의 평균 점수는 cv_results_의 mean_test_score 에 저장

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [44]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # argmax로 best index 저장 후
print(gs.cv_results_['params'][best_index])              # best index를 params 인데스 안에서 찾아서 출력

{'min_impurity_decrease': 0.0001}


In [45]:
# 더 복잡한 매개변수 조합을 탐색해보기
# min_impurity_decrease는 노드를 나누기 위한 불순도 감소 최소량을 지정
# max_depth로 트리의 깊이 제한
# min_samples_split 노드를 나누기 위한 최소 샘플 수 
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001), # 넘파이의 arange 함수 = 0.0001 -> 0.001 까지 0.0001씩 증가하는 배열을 만듬 = 0.001 빼고 총 9개
          'max_depth':range(5, 20, 1),                              # 파이썬의 range 함수(only정수) = 5 -> 20(5~19) 까지 1식 증가하는 배열을 만듬 = 총 15개
          'min_samples_split':range(2, 100, 10)                     # (2~92) 10개
          }

          # 총 9 * 15 * 10 에 5폴드 니까 *5 = 6750개의 모델 수가 만들어짐 

In [46]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [47]:
print(gs.best_params_) # 최상의 매개변수 조합

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [48]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 검증 점수

0.8683865773302731


---
### 랜덤서치
* 매개변수의 값이 수치일 때, 값의 범위나 탐색 간격을 임의로 정하기 어려움
* 또 너무 많은 매개변수 조건이 있어도 그리드 서치 수행 시간이 오래 걸릴 수 있다는 문제 
* 그럴 때 랜덤 서치 방법을 사용하면 좋음

* 랜덤 서치는 탐색하고자하는 매개변수 값의 목록을 전달하는 것이 아니라,
* 매개변수를 샘플링 할 수 있는 확률 분포를 객체를 전달함

In [49]:
from scipy.stats import uniform, randint # scipy 는 적분, 보간, 선형 대수, 확률 등의 수치 계산 전용 라이브러리
# uniform 과 randint 는 모두 주어진 범위에서 고르게 값을 뽑아옴 -> 균등분포에서 샘플링
# randint 는 정수값을
# uniform 은 실수값을 뽑음

In [50]:
rgen = randint(0,10)  # 0~10사이의 숫자중
rgen.rvs(10) # 10개 랜덤 샘플링

array([8, 6, 7, 5, 4, 4, 2, 5, 9, 5])

In [51]:
np.unique(rgen.rvs(1000), return_counts = True)
#  0~9 까지의 숫자를 1000개 샘플링 해서 각 숫자마다 얼마나 뽑혔는지 산출되도록 해보면 고르게 샘플링 된다는 것을 알 수 있음

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 105, 107,  85,  85, 103,  99,  86, 100, 125]))

In [52]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25),}


In [56]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)
# params에서 정의된 매개변수 범위에서 총 100번 샘플링 하여 교차 검증을 수행 -> 최적의 매개변수 조합을 찾음
# 앞에서의 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색 가능

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6561c6e8d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6561c6ee90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6561b531d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6561c6e450>},
                   random_state=42)

In [57]:
print(gs.best_params_) # 최적의 매개변수 조합

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [60]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최고의 교차 검증 점수

0.8695428296438884


In [61]:
dt = gs.best_estimator_ # 최적의 모델은 이미 전체 훈련세트로 train_input train_target으로 훈련되어 best_estimator_ 속성에 저장됨
print(dt.score(test_input, test_target)) # 최종모델로 결정 후 테스트 세트의 성능 확인

0.86


테스트 세트 점수 < 검증세트 보다 조금 작은 것이 일반적임
다양한 매개변수를 테스트 해서 얻은 결과인 점이 중요

### 결론 : 수동으로 매개변수를 바꾸지 말고, 그리드 서치나 랜덤 서치를 사용